# PART1

## 1. ESTUDIO PREVIO DE LOS ARCHIVOS
En este apartado se realizarán comprobaciones para verificar que la información sobre los campos dada en los PDFs es correcta.
Por ejemplo, el archivo 'actividadeseconomicas' puede tener campos repetidos, es decir, que ya se encuentran en el archivo 'locales'. Por lo que sería bastante eficiente eliminar esos campos duplicados, los cuales pueden ser unidos por su clave foranea ID_LOCAL en el futuro.
Esto proporcionaría velocidad de consulta , ahorro de espacio y de computo a futuro.

In [2]:
#------------Dependencies-----------#
import pandas as pd
import numpy as np
import os

In [3]:
#Path to files
path_actividad = "data_files/actividadeconomica202312.csv"
path_licencias = "data_files/licencias202312.csv"
path_locales = "data_files/locales202312.csv"
path_terrazas = "data_files/terrazas202312.csv"


**Dataframe creation 'Actividad Economica'**

In [4]:
df_actividad = pd.read_csv(
    path_actividad,
    sep = ";",
    skip_blank_lines = True,
    on_bad_lines = 'skip',
    engine = 'python',
    encoding = 'utf-8'
)

**Dataframe creation 'Licencias'**

In [5]:
df_licencias = pd.read_csv(path_actividad,sep = ";",skip_blank_lines = True,on_bad_lines = 'skip',engine = 'python',  encoding = 'utf-8')

**Dataframe creation 'Locales'**

In [6]:
df_locales = pd.read_csv(path_locales,sep = ";",skip_blank_lines = True,on_bad_lines = 'skip',engine = 'python',  encoding = 'utf-8')

**Dataframe creation 'Terrazas'**

In [7]:
df_terrazas = pd.read_csv(path_terrazas,sep = ";",skip_blank_lines = True,on_bad_lines = 'skip',engine = 'python',  encoding = 'utf-8')

## 1.1 CAMPOS DUPLICADOS

**Locales vs Actividades**

In [8]:
#Query por 'id_local'
resultado = df_locales.query("id_local == 20000756")
resultado.id_local,resultado.coordenada_y_local,resultado.id_tipo_acceso_local,resultado.id_tipo_acceso_local

(6    20000756
 Name: id_local, dtype: int64,
 6    4472755.5
 Name: coordenada_y_local, dtype: float64,
 6    1
 Name: id_tipo_acceso_local, dtype: int64,
 6    1
 Name: id_tipo_acceso_local, dtype: int64)

In [9]:
#Query por 'id_local'
resultado = df_actividad.query("id_local == 20000756")
resultado.id_local,resultado.coordenada_y_local,resultado.id_tipo_acceso_local,resultado.id_tipo_acceso_local

(6515    20000756
 Name: id_local, dtype: int64,
 6515    4472755.5
 Name: coordenada_y_local, dtype: float64,
 6515    1
 Name: id_tipo_acceso_local, dtype: int64,
 6515    1
 Name: id_tipo_acceso_local, dtype: int64)

In [10]:
#Query por 'id_local'
resultado = df_terrazas.query("id_local == 20000756")
resultado.id_local,resultado.coordenada_y_local,resultado.id_tipo_acceso_local,resultado.id_tipo_acceso_local

(5471    20000756
 Name: id_local, dtype: int64,
 5471    4472755.5
 Name: coordenada_y_local, dtype: float64,
 5471    1
 Name: id_tipo_acceso_local, dtype: int64,
 5471    1
 Name: id_tipo_acceso_local, dtype: int64)

**Como se puede comprobar hay campos que tienen los datos repetidos en ambos archivos por lo que la información descrita en los PDFs es veraz**
Lo que plantearé será dejar el dataframe "locales" como el archivo maestro y luego eliminaré de los demás archivos las dimensiones que posean datos repetidos, así solo quedarán los campos con información limpia y quedarán las futuras colecciones mejor estructuradas.

En el caso en que se quieran realizar consultas de union de tablas el ID **id_local** servirá con clave foranea en las demás tablas.

**Eliminacion de columnas con información duplicadas**

In [11]:
# 1. DataFrame de Locales (fuente principal)
# No hacemos cambios en df_locales

# 2. DataFrame de Actividades
# Mantener solo campos únicos de actividades
columnas_actividades = [
    'id_local',  # clave de unión
    'id_seccion', 'desc_seccion', 'id_division', 'desc_division',
    'id_epigrafe', 'desc_epigrafe'
]
df_actividad = df_actividad[columnas_actividades]

# 3. DataFrame de Licencias
# Mantener solo campos únicos de licencias (pero no aparecen en el índice proporcionado)
# Asumimos que deberían estar estas columnas basadas en los PDFs:
columnas_licencias = [
    'id_local',  # clave de unión
    'ref_licencia', 'id_tipo_licencia', 'desc_tipo_licencia',
    'ide_tipo_situacion_licencia', 'desc_tipo_situacion_licencia',
    'Fecha_Dec_Lic'
]
# Filtramos solo las columnas que existan realmente
columnas_licencias = [col for col in columnas_licencias if col in df_licencias.columns]
df_licencias = df_licencias[columnas_licencias] if columnas_licencias else df_licencias[['id_local']]

# 4. DataFrame de Terrazas
# Mantener solo campos únicos de terrazas
columnas_terrazas = [
    'id_terraza', 'id_local',  # claves
    'Superficie_ES', 'Superficie_RA', 'mesas_aux_es', 'mesas_aux_ra',
    'mesas_es', 'mesas_ra', 'sillas_es', 'sillas_ra', 'hora_ini_LJ_es',
    'hora_fin_LJ_es', 'hora_ini_LJ_ra', 'hora_fin_LJ_ra', 'hora_ini_VS_es',
    'hora_fin_VS_es', 'hora_ini_VS_ra', 'hora_fin_VS_ra',
    'desc_ubicacion_terraza', 'id_periodo_terraza', 'desc_periodo_terraza',
    'id_situacion_terraza', 'desc_situacion_terraza'
]
df_terrazas = df_terrazas[columnas_terrazas]

# Columnas de metadatos que podríamos querer conservar
columnas_metadatos = ['fx_carga', 'fx_datos_ini', 'fx_datos_fin']

# Añadir metadatos a cada DataFrame si existen
for df in [df_actividad, df_licencias, df_terrazas]:
    for col in columnas_metadatos:
        if col in df.columns:
            df[col] = df[col]

## 1.2 Limpieza y análisis de los DF

**DF_LOCALES**

In [12]:
df_locales.describe()

,id_local,id_distrito_local,id_barrio_local,cod_barrio_local,id_seccion_censal_local,desc_seccion_censal_local,coordenada_x_local,coordenada_y_local,id_tipo_acceso_local,id_situacion_local,...,secuencial_local_PC,id_vial_acceso,id_ndp_acceso,id_clase_ndp_acceso,num_acceso,coordenada_x_agrupacion,coordenada_y_agrupacion,id_agrupacion,id_tipo_agrup,cod_postal
count,1.511620e+05,151162.000000,151162.000000,151162.000000,151162.000000,151162.000000,151162.000000,1.511620e+05,151162.000000,151162.000000,...,151162.000000,1.511620e+05,1.511620e+05,151162.0,151162.000000,14501.000000,1.450100e+04,1.511620e+05,151162.000000,151162.000000
mean,2.655094e+08,9.939661,997.419001,3.452925,10011.069158,71.408396,399836.581200,4.049711e+06,1.096982,2.287493,...,22.690491,5.615678e+06,1.473663e+07,1.0,42.168978,393972.972789,3.984938e+06,9.497111e+06,-0.224084,28025.110689
std,5.016712e+07,5.625185,562.402277,1.862671,5626.998295,48.532575,129534.909558,1.311569e+06,1.486127,2.003236,...,65.050187,1.979993e+07,6.694123e+06,0.0,94.405255,138115.023079,1.396506e+06,2.915522e+07,2.928384,14.277943
min,1.000000e+07,1.000000,101.000000,1.000000,1001.000000,1.000000,0.000000,0.000000e+00,0.000000,1.000000,...,0.000000,1.270000e+02,1.100000e+07,1.0,1.000000,0.000000,0.000000e+00,-1.000000e+00,-1.000000,28001.000000
25%,2.704840e+08,5.000000,505.000000,2.000000,5085.000000,32.000000,439092.600000,4.470752e+06,1.000000,1.000000,...,10.000000,2.244000e+05,1.103422e+07,1.0,8.000000,438973.620000,4.469252e+06,-1.000000e+00,-1.000000,28014.000000
50%,2.800378e+08,10.000000,1004.000000,3.000000,10137.000000,65.000000,440758.600000,4.473638e+06,1.000000,1.000000,...,10.000000,4.322000e+05,1.108596e+07,1.0,20.000000,441296.600000,4.473220e+06,-1.000000e+00,-1.000000,28025.000000
75%,2.850114e+08,15.000000,1501.000000,5.000000,15041.000000,104.000000,443691.620000,4.476700e+06,1.000000,4.000000,...,20.000000,6.511000e+05,2.000951e+07,1.0,46.000000,444449.560000,4.477306e+06,-1.000000e+00,-1.000000,28037.000000
max,3.000220e+08,21.000000,2105.000000,9.000000,21034.000000,222.000000,454736.530000,4.495496e+06,12.000000,10.000000,...,9010.000000,8.895311e+07,3.107177e+07,1.0,5090.000000,452667.560000,4.488548e+06,9.900073e+07,17.000000,28055.000000
